<a href="https://colab.research.google.com/github/workingbetter/ITNPAI1/blob/main/ITNPAI1_3071806.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

#University of Stirling - Spring 2023

## ITNPAI1 - Deep Learning for Vision and NLP (2022/3)

---

# ITNPAI1 Project
MICHAEL BOGALE
3071806

# 1. Problem Defination
## Image Classification problem
Creating a deep learning model that classifies cars and trees on image dataset collected from Areal imaging on Google Earth Pro and testing its performance.
## Dataset collected 
Addis Ababa 

Stirling




# 2. Github repository
repository created

###Invitation sent usernames

1. sandyCarmichael
2. vak000
3. jeferssonalex

# 3. Dataset Creation
##Created dataset containing 400 images on each subset
## Addis Ababa Labeled Images
####Addis Ababa Car (400)
####Addis Ababa Tree (400)
####Addis Ababa Neither (400)
## Stirling Labeled Images
####Stirling Car (400)
####Stirling Tree (400)
####Stirling Neither (400)




Creating labeling Dataframe and also extracting it to csv file for easier access

In [ ]:
import os
import pandas as pd
"""the dataset is locate on my drive therefore it cant be accessed directly from github
may be i will find a way to access the zip file in github """

# Directory path Image dataset
stir_path = "/content/drive/MyDrive/Deep learning/dataset/Stirling Labeled/"
addis_path = "/content/drive/MyDrive/Deep learning/dataset/Addis Ababa Labeled/"

# Defining a function for generating dataframe of lables 
def labeler_csv(path):
  data = []
  for folder in os.listdir(path):
      for file in os.listdir(os.path.join(path, folder)):
        if file != '.DS_Store':
          data.append([os.path.join(folder, file), folder])
  df = pd.DataFrame(data, columns=['filename', 'label'])
  # df.to_csv('/content/drive/MyDrive/Stirling_labels.csv', index=False)
  return(df)

print(labeler_csv(stir_path))

                   filename label
0       car/image1_1x13.jpg   car
1       car/image1_3x10.jpg   car
2       car/image1_3x11.jpg   car
3        car/image1_7x1.jpg   car
4       car/image1_7x29.jpg   car
...                     ...   ...
1195   tree/image9_8x30.jpg  tree
1196  tree/image10_1x13.jpg  tree
1197  tree/image10_1x11.jpg  tree
1198  tree/image10_1x14.jpg  tree
1199  tree/image10_1x16.jpg  tree

[1200 rows x 2 columns]
